In [3]:
import json

In [2]:
data = '{"dateAndTime":"2024-10-15T12:11","sidewalkPresence":true,"sidewalkWidth":{"value":"Midsize (1.5m - 1.8m)","remarks":null},"sidewalkCondition":{"value":"Cracked (Small cracks <6mm width)","remarks":null},"rampGradient":{"value":"> 5% Gradient (Unaccessible)","remarks":null},"streetFurniture":{"value":"No","remarks":null},"borderBuffer":{"value":"No","remarks":null},"lightingCondition":{"value":"Poor","remarks":null}}'

In [3]:
parsed_data = json.loads(data)

In [4]:
parsed_data

{'dateAndTime': '2024-10-15T12:11',
 'sidewalkPresence': True,
 'sidewalkWidth': {'value': 'Midsize (1.5m - 1.8m)', 'remarks': None},
 'sidewalkCondition': {'value': 'Cracked (Small cracks <6mm width)',
  'remarks': None},
 'rampGradient': {'value': '> 5% Gradient (Unaccessible)', 'remarks': None},
 'streetFurniture': {'value': 'No', 'remarks': None},
 'borderBuffer': {'value': 'No', 'remarks': None},
 'lightingCondition': {'value': 'Poor', 'remarks': None}}

In [1]:
data2 = "{'_state': <django.db.models.base.ModelState at 0x7f9d3073a3c0>,'id': 311,'accessibility_score': Decimal('1.00'),'adjacent_street': '221-17 A. Del Rosario St','data': {'zone': 'City Core Commercial','hazard': 2,'weight': None,'population': 2980,'nearestStreet': '221-17 A. Del Rosario St','nearPlacesFrequency': {'food': 20,'lodging': 1,'restaurant': 20,'establishment': 20,'meal_delivery': 1,'meal_takeaway': 6,'point_of_interest': 20}},'results': '123.94318663887299,10.327099440742616','anchor': '123.94220321636607,10.328668797875494','start_coordinates_id': 311,'end_coordinates_id': 312,'removed': False}"

In [4]:
parsed_data2 = json.loads(data2)

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)

In [6]:
parsed_data['sidewalkWidth']['value']

'Midsize (1.5m - 1.8m)'

In [9]:
import pickle
with open('models/logistic_regression_model.pkl', 'rb') as file:
    model = pickle.load(file)

In [ ]:
from annotation.models import Location
from annotation.utils.fis.fis import FuzzyInferenceSystem
from annotation.utils.logreg.predict import get_probabilities
from annotation.utils.weather import get_weather_data

with open('models/logistic_regression_model.pkl', 'rb') as file:
    model = pickle.load(file)

anchored_weather_data = {}

for coordinates, _ in Location.ANCHOR_CHOICES:
    coordinates = coordinates.split(',')
    longitude = float(coordinates[0])
    latitude = float(coordinates[1])
    weather_data = get_weather_data(latitude, longitude)
    anchored_weather_data[coordinates] = weather_data

location = Location.objects.get(id=23)

annotation_data = location.annotations.all()

annotation_data = annotation_data.first()
annotation_data = json.loads(annotation_data)

weather = weather_data[location.anchor]

input = {
    'flood_risk': 0,
    'heat_index': 0,
    'precipitation': 0,
    'key_areas': 0,
    'population': 0,
    'walkway_width': 0,
    'zone_area': 0,
    'gradient': 0,
    'surface': 0,
    'street_furniture': 0,
    'border_buffer': 0,
    'lighting': 0
}            

fis = FuzzyInferenceSystem(input)

crisp_weather_condition = fis.crisp_weather_condition
crisp_urban_density = fis.crisp_urban_density
crisp_sidewalk_capacity = fis.crisp_sidewalk_capacity
crisp_safety_risk = fis.crisp_safety_risk
accessibility = fis.accessibility

input_data = [[crisp_weather_condition,crisp_urban_density,crisp_sidewalk_capacity,crisp_safety_risk]]

probabilities = get_probabilities(model,input_data)

accessibility_probability = probabilities[0][0]